<a href="https://colab.research.google.com/github/Bindu3sriya/bitcoin_pridction/blob/main/bitcoin_price_prediction_and_notifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime
import requests



In [103]:
data = pd.read_csv('https://raw.githubusercontent.com/Bindu3sriya/dataset/main/BTC-USD.csv')


In [104]:
data.dropna(subset=['Date', 'Close'], inplace=True)
data.isnull().sum()



Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [105]:
data['Date'] = pd.to_datetime(data['Date'])
data['DayOfWeek'] = data['Date'].dt.dayofweek
print(data)


          Date          Open          High           Low         Close  \
0   2022-08-06  23291.423828  23326.562500  22961.279297  22961.279297   
1   2022-08-07  22963.505859  23359.009766  22894.556641  23175.890625   
2   2022-08-08  23179.527344  24203.689453  23176.546875  23809.486328   
3   2022-08-09  23811.484375  23898.615234  22982.000000  23164.318359   
4   2022-08-10  23162.898438  24127.414063  22771.519531  23947.642578   
..         ...           ...           ...           ...           ...   
358 2023-07-30  29357.093750  29443.169922  29059.501953  29275.308594   
359 2023-07-31  29278.314453  29489.873047  29131.578125  29230.111328   
360 2023-08-01  29230.873047  29675.732422  28657.023438  29675.732422   
361 2023-08-02  29704.146484  29987.998047  28946.509766  29151.958984   
365 2023-08-06  29032.826172  29093.957031  28964.210938  29039.011719   

        Adj Close        Volume  DayOfWeek  
0    22961.279297  1.597826e+10          5  
1    23175.890625  1.

In [106]:
train_data, val_data = train_test_split(data, test_size=0.2, shuffle=False)


In [107]:
X_train = train_data[['DayOfWeek']]  # Features
y_train = train_data['Close']         # Target
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)


In [108]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [109]:
X_val = val_data[['DayOfWeek']]
X_val_scaled = scaler.transform(X_val)


In [110]:
y_val_pred = model.predict(X_val_scaled)
val_rmse = mean_squared_error(val_data['Close'], y_val_pred, squared=False)
print(f'Validation RMSE: {val_rmse:.2f}')

Validation RMSE: 6997.93


In [126]:
user_threshold = float(input("Enter your desired price threshold: "))

Enter your desired price threshold: 22049.36


In [127]:
current_day_of_week = datetime.datetime.now().weekday()

In [128]:
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [129]:
notification_message = ""
for day_offset in range(1, 7):  # Predict next 6 days
    next_day = (current_day_of_week + day_offset) % 7
    predicted_price = model.predict([[next_day]])[0]

    if predicted_price <= user_threshold:
        day_name = day_names[next_day]
        day_date = (datetime.datetime.now() + datetime.timedelta(days=day_offset)).strftime('%Y-%m-%d')
        notification_message += f"Predicted price for {day_name} ({day_date}) is below threshold. Predicted price: {predicted_price:.2f} USD.\n"

# Send notification if applicable
if notification_message:
    print("Sending notification...")
    api_url = 'https://api.callmebot.com/whatsapp.php?phone=+919573639109&apikey=2278832&text=' + notification_message
    response = requests.post(api_url)

    if response.status_code == 200:
        print("WhatsApp notification sent successfully.")
    else:
        print("Error sending WhatsApp notification.")
else:
    print("No days with predicted price below threshold.")

print("Prediction and notification complete.")

Sending notification...
WhatsApp notification sent successfully.
Prediction and notification complete.
